Pylbm for solids.

Pylbm is a python-package for numerical simulations using the Lattice-Boltzmann-Method (LBM). For more general information, refer to the website https://pylbm.readthedocs.io/en/latest/, this documentation will focus on the scope of the project, which is to extend and use pylbm to solve equations of quasi-static linear elasticity in 2D. 

In this notebook we will have a look at boundary conditions.

Boundary conditions can be prescribed through the labels we assign to the box and/or elements when defining a geometry. The labels -1 and -2 are reserved internally by pylbm with -1 being used for periodic boundary conditions. Pylbm provides a number of classes for handling boundary conditions which can be seen in `pylbm/boundary.py`, such as Bounce-Back, Anti-Bounce-Back etc. During this project I introduced two new classes for prescribing 2nd Order accurate Neumann and Dirichlet BCs for linear elasticity, called `pylbm.bc.LEDirichlet2D` and `pylbm.bc.LENeumann2D` respectively. BCs can be specified in the dictionary through the `'boundary_conditions'` key:
```python
'boundary_conditions': {1: {'method': {0: pylbm.bc.LEDirichlet2D}, 'value': bc_1},
                        2: {'method': {0: pylbm.bc.LENeumann2D},   'value': bc_2},
                        },
```
In this example we apply Dirichlet BCs to all boundaries with label 1 and Neumann BCs to boundaries with label 2. `bc_1` and `bc_2` are functions which define the values we prescribe on the boundary. Example for defining these functions:
```python
def bc_1(f, m, x, y):
    m[u] = x+y
    m[v] = x*y

def bc_2(f, m, x, y):
    m[u] = x**2-y**2
    m[v] = np.sin(x*y)
```
Pylbm expects either a float or int for `'value'` or a function which takes f (populations), m (moments) and coordinates as inputs. Note that no symbols should be used here. Note also that we prescribe BCs for both x and y direction in the function with `m[u]` and `m[v]` respectively (remember that we used u and v for the conserved moments $m_{10}$ and $m_{01}$). For both the Neumann and Dirichlet implementation, the argument f in the function is unused and only included because pylbm expects it. The boundary values should be prescribed via m. For complicated expressions (especially when we need to involve normal vectors in Neumann BCs) it may prove easier to setup an expression with sympy and 'lambdify' it:
```python
import sympy as sp

T_x = *complicated sympy expression*
T_y = *complicated sympy expression*
T_x_f = sp.lambdify([X,Y],T_x,"numpy")
T_y_f = sp.lambdify([X,Y],T_y,"numpy")

def bc(f, m, x, y):
    m[u] = T_x_f(x,y)
    m[v] = T_y_f(x,y)
```
Below is an example of a plate with hole geometry where we prescribe Neumann BCs for the outside boundary and Dirichlet for the inside.

In [5]:
import sympy as sp
import pylbm
import numpy as np

#geometry
dx = 0.1
dt = 0.1
la = dx/dt
L_box = 1.

#characteristic dimensions
U = 1.
L = dx
T = dt

#geometry in lattice units
dx_nd = dx/L
L_box_nd = L_box/L
la_nd = 1.

#shapes to construct a ring geometry (note that parameters are scaled to lattice units)
shape_1 = pylbm.Parallelogram((0,0), (1/L, 0), (0, 1/L), label=1, isfluid=True)
shape_2 = pylbm.Circle((0.5/L,0.5/L), 0.25/L, label=2, isfluid=False)

#material parameters (actual values and in lattice units)
E = 0.1/0.1731
E_nd = T/(L*L)*E
nu = .7
K = E/(2*(1-nu))
K_nd = K*T/L**2
mu = E/(2*(1+nu))
mu_nd = mu*T/L**2
theta = 1/3

#define symbols
u, v, LA = sp.symbols('u, v, LA')
THETA, MU_ND, K_ND, GAMMA = sp.symbols('THETA, MU_ND, K_ND, GAMMA')

#moment matrix
M = sp.Matrix([[1,0,-1,0,1,-1,-1,1],[0,1,0,-1,1,1,-1,-1],[0,0,0,0,1,-1,1,-1],[1,1,1,1,2,2,2,2],
    [1,-1,1,-1,0,0,0,0],[0,0,0,0,1,-1,-1,1],[0,0,0,0,1,1,-1,-1],[GAMMA,GAMMA,GAMMA,GAMMA,1+2*GAMMA,1+2*GAMMA,1+2*GAMMA,1+2*GAMMA]])

#equilibrium moments in order
Meq = [u,v,0,0,0,THETA*u,THETA*v,0]

#relaxation parameters
w10 = 0.
w01 = 0.
w11 = 1/(MU_ND/THETA+.5)
ws = 1/(2*K_ND/(1+THETA)+.5)
wd = 1/(2*MU_ND/(1-THETA)+.5)
w12 = 1.5
w21 = 1.5
wf = 1.
omega = [w10,w01,w11,ws,wd,w12,w21,wf]
gamma = theta*.5/((1+theta)*((1/ws-.5).evalf(subs={K_ND:K_nd,THETA:theta})-.5))

#boundary conditions
def bc_1(f, m, x, y):
    m[u] = 1e-7*((x*L)**2-(y*L)**2)
    m[v] = 1e-7*np.sin(x*y*L**2)
    
def bc_2(f, m, x, y):
    m[u] = 0.
    m[v] = 0.

#initial condition
def u_init(x,y):
    return 0.

def v_init(x,y):
    return 0.

dico = {
    'box' : {'x': [0,L_box_nd], 'y': [0,L_box_nd], 'label': 1},
    'elements': [shape_1, shape_2],
    'space_step': dx_nd,
    'scheme_velocity': LA,
    'parameters': {LA: la_nd,
            THETA: theta,
            MU_ND: mu_nd,
            K_ND: K_nd,
            GAMMA: gamma,
            },
    'init': {u: u_init, v: v_init},
    'boundary_conditions': {1: {'method': {0: pylbm.bc.LENeumann2D}, 'value': bc_1},
                            2: {'method': {0: pylbm.bc.LEDirichlet2D}, 'value': bc_2},
                            },
    'generator': 'cython',
    'lbm_algorithm': {'name': pylbm.algorithm.BaseAlgorithm},
    'schemes':[
            {
                'velocities': list(range(1,9)),
                'conserved_moments': [u,v],
                'M': M,
                'equilibrium': Meq,
                'relaxation_parameters': omega,
                'source_terms': {u: 0., v: 0.}
                }]
    }

In [6]:
#create and run a simulation 
sim = pylbm.Simulation(dico)
for i in range(1000):
    sim.one_time_step()